In [1]:
import pandas as pd
import numpy as np
from vosk import Model, KaldiRecognizer, SetLogLevel
import sys
import os
import wave
import subprocess
import json
from os import listdir
from os.path import isfile, join
import datetime
import time

In [2]:
start = time.time()

files=pd.DataFrame(columns=['folder','path_audio'])

dir_audios="../Data/Audio"
dir_audios= dir_audios+"/" if dir_audios[-1]!="/" else dir_audios
folders_audio = [dir_audios+f for f in listdir(dir_audios)]

for folders in folders_audio:
    file_audio = [folders+"/"+f for f in listdir(folders)]
    file_audio = pd.DataFrame(file_audio,columns=['path_audio'])
    file_audio['folder'] = folders.split("/")[-1]

    files = files.append(file_audio)
files.head()

,folder,path_audio
0,C5N,../Data/Audio/C5N/C5N2020-08-15-17-00-45.wav
1,C5N,../Data/Audio/C5N/C5N2020-08-17-17-54-34.wav
2,C5N,../Data/Audio/C5N/C5N2020-08-25-17-55-42.wav
3,C5N,../Data/Audio/C5N/C5N2020-08-18-17-57-53.wav
4,C5N,../Data/Audio/C5N/C5N2020-08-14-17-55-49.wav


In [3]:
files['file']=files['path_audio'].apply(lambda x: x.split("/")[-1])
aux = pd.Series(files['file'].apply(lambda x: x.replace(".wav","").split("-")[-6:]))
aux = aux.apply(lambda x: [int(a) if len(a)<=4 else int(a[-4:]) for a in x])
files['date'] = aux.apply(lambda x: datetime.datetime(x[0], x[1], x[2], hour=x[3], minute=x[4], second=x[5]))

files.reset_index(drop=True, inplace=True)
files

,folder,path_audio,file,date
0,C5N,../Data/Audio/C5N/C5N2020-08-15-17-00-45.wav,C5N2020-08-15-17-00-45.wav,2020-08-15 17:00:45
1,C5N,../Data/Audio/C5N/C5N2020-08-17-17-54-34.wav,C5N2020-08-17-17-54-34.wav,2020-08-17 17:54:34
2,C5N,../Data/Audio/C5N/C5N2020-08-25-17-55-42.wav,C5N2020-08-25-17-55-42.wav,2020-08-25 17:55:42
3,C5N,../Data/Audio/C5N/C5N2020-08-18-17-57-53.wav,C5N2020-08-18-17-57-53.wav,2020-08-18 17:57:53
4,C5N,../Data/Audio/C5N/C5N2020-08-14-17-55-49.wav,C5N2020-08-14-17-55-49.wav,2020-08-14 17:55:49
5,TN,../Data/Audio/TN/TN2020-08-15-16-30-30.wav,TN2020-08-15-16-30-30.wav,2020-08-15 16:30:30
6,TN,../Data/Audio/TN/TN2020-08-17-19-00-08.wav,TN2020-08-17-19-00-08.wav,2020-08-17 19:00:08
7,TN,../Data/Audio/TN/TN2020-08-25-18-59-36.wav,TN2020-08-25-18-59-36.wav,2020-08-25 18:59:36
8,TN,../Data/Audio/TN/TN2020-08-14-18-59-25.wav,TN2020-08-14-18-59-25.wav,2020-08-14 18:59:25
9,TN,../Data/Audio/TN/TN2020-08-18-19-00-19.wav,TN2020-08-18-19-00-19.wav,2020-08-18 19:00:19


In [ ]:
folder_model="../Data/Modelos/audio_model_spanish_vosk"
model = Model(folder_model)

registros = pd.DataFrame(columns=['path_audio','result','text'])
i=0
for file in files['path_audio']:
    i=i+1
    print(i,"- Procesando archivo",file)
    wf = wave.open(file, "rb")
    rec = KaldiRecognizer(model, wf.getframerate())    

    while True:
        data = wf.readframes(1000000)
        if len(data) == 0:
            break
        if rec.AcceptWaveform(data):
            try:
                new_input = json.loads(rec.Result())
                new_input = pd.DataFrame(new_input)
                new_input['path_audio']=file
                registros = registros.append(new_input)
            except ValueError:
                 continue
        else:
            rec.PartialResult()


1 - Procesando archivo ../Data/Audio/C5N/C5N2020-08-15-17-00-45.wav
2 - Procesando archivo ../Data/Audio/C5N/C5N2020-08-17-17-54-34.wav
3 - Procesando archivo ../Data/Audio/C5N/C5N2020-08-25-17-55-42.wav
4 - Procesando archivo ../Data/Audio/C5N/C5N2020-08-18-17-57-53.wav
5 - Procesando archivo ../Data/Audio/C5N/C5N2020-08-14-17-55-49.wav
6 - Procesando archivo ../Data/Audio/TN/TN2020-08-15-16-30-30.wav


In [ ]:
registros.reset_index(drop=True,inplace=True)
registros_2 = registros.join(pd.DataFrame(x for x in registros['result']))
registros_2.drop(columns='result',inplace=True)
registros_2.head()

In [ ]:
MinTime = pd.DataFrame(registros_2.groupby(['path_audio','text'])['start'].min())
MaxTime = pd.DataFrame(registros_2.groupby(['path_audio','text'])['end'].max())

registros_3=registros_2[['path_audio','text']]
registros_3 = registros_3.merge(MinTime,on=['path_audio','text'])
registros_3 = registros_3.merge(MaxTime,on=['path_audio','text'])

registros_3.drop_duplicates(inplace=True)
registros_3.reset_index(drop=True,inplace=True)


registros_3.head()

In [ ]:
data = files.merge(registros_3, on='path_audio')

data['start'] = data['start'] .apply(lambda x: datetime.timedelta(seconds=int(x)))
data['start'] = data['start'] + data['date']
data['end'] = data['end'] .apply(lambda x: datetime.timedelta(seconds=int(x)))
data['end'] = data['end'] + data['date']

print(data.shape)
data.head()

In [ ]:
data_text_complete  = pd.DataFrame(columns=['path_audio','text_complete'])
for file in data['path_audio'].unique():
    text = data.loc[data['path_audio'] == file,'text'].unique()
    text_complete = ','.join(text)
    new_input=pd.DataFrame([text_complete], columns=['text_complete'])
    new_input['path_audio'] = file
    data_text_complete = data_text_complete.append(new_input)

data_text_complete.reset_index(drop=True,inplace=True)
data_text_complete=data[['folder','path_audio','file','date']].drop_duplicates().merge(data_text_complete, on = 'path_audio')

In [ ]:
pd.set_option('display.max_colwidth',50) ##el segundo parametro indica la cantidad de caracteres a mostrar, con valor -1 muestra todo
data_text_complete

In [ ]:
##Export datasets
data.to_csv("../Data procesada/data_audio.csv")
data_text_complete.to_csv("../Data procesada/data_audio_concatenada.csv")

In [ ]:
end = time.time()
print("Tiempo de procesamiento:",round((end - start)/60),"minutos")